In [1]:
from torch import nn, optim
from torchvision import transforms, datasets
from torchvision.utils import save_image

import numpy as np
import imageio as img
import math
import time
import os

In [2]:
# models
from lib import vgan

# helper functions
from lib.helper import *

In [3]:
# unlike tensorflow where we have to define the graphs and their operations, in pytorch we deine the network as a class
# i.e we just define the operations and the forward prop, the backprop is sorta in built

# we get this done by defining our class that inherits fromthe torchnn.module and hence the other functions
# such as backprop can be used

In [4]:
print("Status of GPU: ", torch.cuda.is_available())

Status of GPU:  True


In [5]:
# Hyper Parameters
USE_CUDA = True 
MODEL_TYPE = "DCGAN"

DATA_FOLDER = "./alpha"
DATA_NAME = DATA_FOLDER.split('/')[-1].upper()

# hyper parameters
bSize = 100
learning_rate = 0.0002

dwidth = dheight = 64
features = dwidth*dheight

# DCGAN params
channels = 1
img_size = dwidth
ndf = ngf = img_size
latent_dim = 100

# Number of steps to apply to the discriminator
d_steps = 1  # In Goodfellow et. al 2014 this variable is assigned to 1

# Number of epochs
num_epochs = 200

In [6]:
# custom logger from (https://raw.githubusercontent.com/diegoalejogm/gans/master/utils.py) 
from lib.utils import Logger

# Create logger instance
logger = Logger(model_name=MODEL_TYPE, data_name=DATA_NAME, cuda=USE_CUDA)

# Discriminator

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is ``(nc) x 64 x 64``
            nn.Conv2d(channels, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# Generator Network

In [8]:
# https://arxiv.org/pdf/1511.06434, DCGAN
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.kernal_size = 4
        self.stride = 2
        self.padding = 1
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( latent_dim, ngf * 8, 
                               kernel_size=self.kernal_size, 
                               stride=1, 
                               padding=0, 
                               bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 
                               kernel_size=self.kernal_size, 
                               stride=self.stride, 
                               padding=self.padding, 
                               bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 
                               kernel_size=self.kernal_size, 
                               stride=self.stride, 
                               padding=self.padding, 
                               bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 
                               kernel_size=self.kernal_size, 
                               stride=self.stride, 
                               padding=self.padding, 
                               bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, channels, 
                               kernel_size=self.kernal_size, 
                               stride=self.stride, 
                               padding=self.padding, 
                               bias=False),
            nn.Tanh()
            
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

# Train Functions

In [9]:
def to_cuda(thing):
    if torch.cuda.is_available() and USE_CUDA: 
        return thing.cuda()
    return thing

In [10]:
def train_discriminator(optimizer, real_data, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, to_cuda(real_data_target(real_data.size(0))))
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, to_cuda(fake_data_target(real_data.size(0))))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error
    return error_real + error_fake, prediction_real, prediction_fake


In [11]:
def train_generator(optimizer, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    
    # Calculate error and backpropagate
    error = loss(prediction, to_cuda(real_data_target(prediction.size(0))))
    error.backward()
    
    # Update weights with gradients
    optimizer.step()
    
    # Return error
    return error

# Data Processing

In [12]:
def get_custom_data(data_folder):
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Resize((dwidth, dheight)),
         transforms.Grayscale(num_output_channels=1),
         transforms.Normalize((.5), (.5))
        ])

    return datasets.ImageFolder(root=data_folder, transform=compose)

In [13]:
# normalize data (https://github.com/soumith/ganhacks)
def mnist_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Resize((dwidth, dheight)),
         transforms.Normalize((.5), (.5))
        ])
    out_dir = '{}/dataset'.format(DATA_FOLDER)
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)

In [14]:
# Load data
# data = mnist_data()

# custom data
folder = "./alpha/AmplifiedDataset"
data = get_custom_data(folder)

In [15]:
# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=bSize, shuffle=True)
# Num batches
num_batches = len(data_loader)

# Defining Network Essentials

In [16]:
# Vanilla GAN
# discriminator = to_cuda(vgan.Discriminator(features))
# generator = to_cuda(vgan.Generator(bSize, features))

# DCGAN
discriminator = to_cuda(Discriminator())
generator = to_cuda(Generator())

In [17]:
# Optimizers
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)


# If we replace vᵢ = D(xᵢ) and yᵢ=1 ∀ i (for all i) in the BCE-Loss definition, 
# we obtain the loss related to the real-images. Conversely if we set vᵢ = D(G(zᵢ)) and yᵢ=0 ∀ i, \
# we obtain the loss related to the fake-images. 
# In the mathematical model of a GAN I described earlier, the gradient of this had to be ascended, 
# but PyTorch and most other Machine Learning frameworks usually minimize functions instead. 
# Since maximizing a function is equivalent to minimizing it’s negative, and the BCE-Loss term has a minus sign, 
# we don’t need to worry about the sign.
# Loss function
loss = nn.BCELoss()

# Actual Training

In [18]:
# for testing Vanilla GAN
num_test_samples = 16
test_noise = noise(num_test_samples, bSize)


# for epoch in range(num_epochs):
#     for n_batch, (real_batch,_) in enumerate(data_loader):
#         N = real_batch.size(0)
#         # 1. Train Discriminator
#         real_data = to_cuda(Variable(images_to_vectors(real_batch, features)))
        
#         # Generate fake data and detach 
#         # (so gradients are not calculated for generator)
#         # can use.clone() as well but it will be a copy
#         # .detach() uses the same memory
#         fake_data = generator(to_cuda(noise(N, bSize))).detach()
        
#         # Train D
#         d_error, d_pred_real, d_pred_fake = \
#               train_discriminator(d_optimizer, real_data, fake_data)

#         # 2. Train Generator
#         # Generate fake data
#         fake_data = generator(to_cuda(noise(N, bSize)))
#         # Train G
        
#         g_error = train_generator(g_optimizer, fake_data)
#         # Log batch error
#         logger.log(d_error, g_error, epoch, n_batch, num_batches)
        
        
#         # Display Progress every few batches
#         if (n_batch) % 100 == 0: 
#             test_images = to_cuda(vectors_to_images(generator(test_noise), 1, dwidth, dheight))

#             logger.log_images(
#                 test_images, num_test_samples, 
#                 epoch, n_batch, num_batches
#             );
#             # Display status Logs
#             logger.display_status(
#                 epoch, num_epochs, n_batch, num_batches,
#                 d_error, g_error, d_pred_real, d_pred_fake
#             )

In [19]:
print(discriminator)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [20]:
print(generator)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [ ]:
# for testing DCGAN
num_test_samples = 12
test_noise = to_cuda(Variable(torch.Tensor(np.random.normal(0, 1, (bSize, latent_dim, 1, 1)))))

for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(data_loader):

        # Adversarial ground truths
        
        valid = to_cuda(Variable(torch.Tensor(imgs.shape[0], 1, 1, 1).fill_(1.0), requires_grad=False))
        fake = to_cuda(Variable(torch.Tensor(imgs.shape[0], 1, 1, 1).fill_(0.0), requires_grad=False))

        # Configure input

        real_imgs = to_cuda(Variable(imgs.type(torch.Tensor)))
   
        # -----------------
        #  Train Generator
        # -----------------

        g_optimizer.zero_grad()

        # Sample noise as generator input
        z = to_cuda(Variable(torch.Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim, 1, 1)))))

        # Generate a batch of images
        gen_imgs = generator(z)
        
        # Loss measures generator's ability to fool the discriminator
        g_loss = loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        g_optimizer.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        d_optimizer.zero_grad()
        # Measure discriminator's ability to classify real from generated samples
        real_loss = loss(discriminator(real_imgs), valid)
        fake_loss = loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        d_optimizer.step()


        # Display Progress every few batches
        n_batch = epoch * len(data_loader) + i
        if (n_batch) % 100 == 0: 
            test_images = vectors_to_images(generator(test_noise), 1, dwidth, dheight)

#             logger.log_images(
#                 test_images, num_test_samples, 
#                 epoch, n_batch, num_batches
#             );
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, i, len(data_loader),
                d_loss, g_loss, real_loss, fake_loss
            )
            save_image(gen_imgs.data[:25], "dc-images/%d.png" % n_batch, nrow=5, normalize=True)

Epoch: [0/200], Batch Num: [0/1335]
Discriminator Loss: 0.7686, Generator Loss: 0.7478
D(x): 0.8646, D(G(z)): 0.6726
Epoch: [0/200], Batch Num: [100/1335]
Discriminator Loss: 0.0007, Generator Loss: 6.9319
D(x): 0.0005, D(G(z)): 0.0010
Epoch: [0/200], Batch Num: [200/1335]
Discriminator Loss: 0.0002, Generator Loss: 8.3806
D(x): 0.0002, D(G(z)): 0.0002
Epoch: [0/200], Batch Num: [300/1335]
Discriminator Loss: 0.1181, Generator Loss: 2.1803
D(x): 0.0005, D(G(z)): 0.2357
Epoch: [0/200], Batch Num: [400/1335]
Discriminator Loss: 0.0030, Generator Loss: 6.1239
D(x): 0.0016, D(G(z)): 0.0044


Epoch: [20/200], Batch Num: [900/1335]
Discriminator Loss: 0.0181, Generator Loss: 6.2439
D(x): 0.0298, D(G(z)): 0.0065
Epoch: [20/200], Batch Num: [1000/1335]
Discriminator Loss: 0.0023, Generator Loss: 7.9438
D(x): 0.0037, D(G(z)): 0.0010
Epoch: [20/200], Batch Num: [1100/1335]
Discriminator Loss: 0.0056, Generator Loss: 5.9021
D(x): 0.0013, D(G(z)): 0.0099
Epoch: [20/200], Batch Num: [1200/1335]
Discriminator Loss: 0.0078, Generator Loss: 7.0037
D(x): 0.0116, D(G(z)): 0.0040
Epoch: [20/200], Batch Num: [1300/1335]
Discriminator Loss: 0.0055, Generator Loss: 7.2955
D(x): 0.0056, D(G(z)): 0.0054
Epoch: [21/200], Batch Num: [65/1335]
Discriminator Loss: 0.0043, Generator Loss: 7.2357
D(x): 0.0062, D(G(z)): 0.0023
Epoch: [21/200], Batch Num: [165/1335]
Discriminator Loss: 0.0501, Generator Loss: 6.3967
D(x): 0.0789, D(G(z)): 0.0214
Epoch: [21/200], Batch Num: [265/1335]
Discriminator Loss: 0.0478, Generator Loss: 4.3806
D(x): 0.0166, D(G(z)): 0.0791
Epoch: [21/200], Batch Num: [365/1335

Epoch: [25/200], Batch Num: [1125/1335]
Discriminator Loss: 0.0061, Generator Loss: 5.8599
D(x): 0.0006, D(G(z)): 0.0117
Epoch: [25/200], Batch Num: [1225/1335]
Discriminator Loss: 0.0033, Generator Loss: 6.5058
D(x): 0.0013, D(G(z)): 0.0054
Epoch: [25/200], Batch Num: [1325/1335]
Discriminator Loss: 0.0049, Generator Loss: 5.7825
D(x): 0.0007, D(G(z)): 0.0092
Epoch: [26/200], Batch Num: [90/1335]
Discriminator Loss: 0.0030, Generator Loss: 8.1791
D(x): 0.0053, D(G(z)): 0.0008
Epoch: [26/200], Batch Num: [190/1335]
Discriminator Loss: 0.0091, Generator Loss: 5.2512
D(x): 0.0011, D(G(z)): 0.0170
Epoch: [26/200], Batch Num: [290/1335]
Discriminator Loss: 0.0026, Generator Loss: 8.3125
D(x): 0.0031, D(G(z)): 0.0021
Epoch: [26/200], Batch Num: [390/1335]
Discriminator Loss: 0.0041, Generator Loss: 6.2691
D(x): 0.0022, D(G(z)): 0.0059
Epoch: [26/200], Batch Num: [490/1335]
Discriminator Loss: 0.0047, Generator Loss: 6.7568
D(x): 0.0024, D(G(z)): 0.0070
Epoch: [26/200], Batch Num: [590/1335]

Epoch: [31/200], Batch Num: [15/1335]
Discriminator Loss: 0.0074, Generator Loss: 6.9722
D(x): 0.0017, D(G(z)): 0.0131
Epoch: [31/200], Batch Num: [115/1335]
Discriminator Loss: 0.0056, Generator Loss: 5.9451
D(x): 0.0016, D(G(z)): 0.0095
Epoch: [31/200], Batch Num: [215/1335]
Discriminator Loss: 0.0033, Generator Loss: 6.6603
D(x): 0.0011, D(G(z)): 0.0055
Epoch: [31/200], Batch Num: [315/1335]
Discriminator Loss: 0.0032, Generator Loss: 6.6289
D(x): 0.0005, D(G(z)): 0.0060
Epoch: [31/200], Batch Num: [415/1335]
Discriminator Loss: 0.0013, Generator Loss: 7.9720
D(x): 0.0016, D(G(z)): 0.0010
Epoch: [31/200], Batch Num: [515/1335]
Discriminator Loss: 0.0015, Generator Loss: 7.3926
D(x): 0.0006, D(G(z)): 0.0023
Epoch: [31/200], Batch Num: [615/1335]
Discriminator Loss: 0.0267, Generator Loss: 8.3294
D(x): 0.0408, D(G(z)): 0.0125
Epoch: [31/200], Batch Num: [715/1335]
Discriminator Loss: 0.0372, Generator Loss: 8.2548
D(x): 0.0722, D(G(z)): 0.0022
Epoch: [31/200], Batch Num: [815/1335]
Di

Epoch: [36/200], Batch Num: [240/1335]
Discriminator Loss: 0.0026, Generator Loss: 6.8084
D(x): 0.0003, D(G(z)): 0.0049
Epoch: [36/200], Batch Num: [340/1335]
Discriminator Loss: 0.0059, Generator Loss: 6.9041
D(x): 0.0074, D(G(z)): 0.0044
Epoch: [36/200], Batch Num: [440/1335]
Discriminator Loss: 0.0086, Generator Loss: 7.9311
D(x): 0.0157, D(G(z)): 0.0015
Epoch: [36/200], Batch Num: [540/1335]
Discriminator Loss: 0.0137, Generator Loss: 5.2424
D(x): 0.0003, D(G(z)): 0.0271
Epoch: [36/200], Batch Num: [640/1335]
Discriminator Loss: 0.0109, Generator Loss: 7.3207
D(x): 0.0186, D(G(z)): 0.0032
Epoch: [36/200], Batch Num: [740/1335]
Discriminator Loss: 0.0012, Generator Loss: 10.4048
D(x): 0.0023, D(G(z)): 0.0002
Epoch: [36/200], Batch Num: [840/1335]
Discriminator Loss: 0.0023, Generator Loss: 7.7573
D(x): 0.0028, D(G(z)): 0.0017
Epoch: [36/200], Batch Num: [940/1335]
Discriminator Loss: 0.0019, Generator Loss: 7.5121
D(x): 0.0013, D(G(z)): 0.0025
Epoch: [36/200], Batch Num: [1040/1335]

Epoch: [41/200], Batch Num: [465/1335]
Discriminator Loss: 0.0552, Generator Loss: 6.8181
D(x): 0.0281, D(G(z)): 0.0823
Epoch: [41/200], Batch Num: [565/1335]
Discriminator Loss: 0.0186, Generator Loss: 7.4286
D(x): 0.0341, D(G(z)): 0.0031
Epoch: [41/200], Batch Num: [665/1335]
Discriminator Loss: 0.0264, Generator Loss: 6.9242
D(x): 0.0431, D(G(z)): 0.0096
Epoch: [41/200], Batch Num: [765/1335]
Discriminator Loss: 0.0091, Generator Loss: 5.4279
D(x): 0.0024, D(G(z)): 0.0157
Epoch: [41/200], Batch Num: [865/1335]
Discriminator Loss: 0.0032, Generator Loss: 7.6900
D(x): 0.0046, D(G(z)): 0.0018
Epoch: [41/200], Batch Num: [965/1335]
Discriminator Loss: 0.0052, Generator Loss: 8.2775
D(x): 0.0090, D(G(z)): 0.0013
Epoch: [41/200], Batch Num: [1065/1335]
Discriminator Loss: 0.0008, Generator Loss: 9.2164
D(x): 0.0011, D(G(z)): 0.0006
Epoch: [41/200], Batch Num: [1165/1335]
Discriminator Loss: 0.0015, Generator Loss: 6.9511
D(x): 0.0001, D(G(z)): 0.0029
Epoch: [41/200], Batch Num: [1265/1335

Epoch: [46/200], Batch Num: [690/1335]
Discriminator Loss: 0.0006, Generator Loss: 8.8085
D(x): 0.0007, D(G(z)): 0.0005
Epoch: [46/200], Batch Num: [790/1335]
Discriminator Loss: 0.0385, Generator Loss: 4.7089
D(x): 0.0005, D(G(z)): 0.0765
Epoch: [46/200], Batch Num: [890/1335]
Discriminator Loss: 1.6970, Generator Loss: 0.5058
D(x): 0.0002, D(G(z)): 3.3939
Epoch: [46/200], Batch Num: [990/1335]
Discriminator Loss: 0.0058, Generator Loss: 6.6047
D(x): 0.0031, D(G(z)): 0.0084
Epoch: [46/200], Batch Num: [1090/1335]
Discriminator Loss: 0.0105, Generator Loss: 7.8534
D(x): 0.0194, D(G(z)): 0.0017
Epoch: [46/200], Batch Num: [1190/1335]
Discriminator Loss: 0.0101, Generator Loss: 7.7790
D(x): 0.0185, D(G(z)): 0.0018
Epoch: [46/200], Batch Num: [1290/1335]
Discriminator Loss: 0.0106, Generator Loss: 6.6090
D(x): 0.0159, D(G(z)): 0.0053
Epoch: [47/200], Batch Num: [55/1335]
Discriminator Loss: 0.0024, Generator Loss: 6.7175
D(x): 0.0002, D(G(z)): 0.0046
Epoch: [47/200], Batch Num: [155/1335]

Epoch: [51/200], Batch Num: [915/1335]
Discriminator Loss: 0.0184, Generator Loss: 4.8706
D(x): 0.0014, D(G(z)): 0.0353
Epoch: [51/200], Batch Num: [1015/1335]
Discriminator Loss: 0.0106, Generator Loss: 5.2207
D(x): 0.0001, D(G(z)): 0.0212
Epoch: [51/200], Batch Num: [1115/1335]
Discriminator Loss: 0.0013, Generator Loss: 7.3322
D(x): 0.0003, D(G(z)): 0.0023
Epoch: [51/200], Batch Num: [1215/1335]
Discriminator Loss: 0.0010, Generator Loss: 10.0173
D(x): 0.0018, D(G(z)): 0.0002
Epoch: [51/200], Batch Num: [1315/1335]
Discriminator Loss: 0.0006, Generator Loss: 8.4791
D(x): 0.0005, D(G(z)): 0.0007
Epoch: [52/200], Batch Num: [80/1335]
Discriminator Loss: 0.0035, Generator Loss: 6.3545
D(x): 0.0007, D(G(z)): 0.0063
Epoch: [52/200], Batch Num: [180/1335]
Discriminator Loss: 0.0048, Generator Loss: 6.3411
D(x): 0.0012, D(G(z)): 0.0084
Epoch: [52/200], Batch Num: [280/1335]
Discriminator Loss: 0.0004, Generator Loss: 10.4682
D(x): 0.0006, D(G(z)): 0.0002
Epoch: [52/200], Batch Num: [380/13